In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from public_data.models import Commune, OcsgeDiff, CommuneDiff, Ocsge, Departement, CommuneSol, CouvertureSol
from project.models import Project

from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from django.db.models import Sum, Q, Value
from django.db.models.functions import Concat

from prettytable import PrettyTable

In [2]:
city = Commune.objects.get(name="Arcachon")
city.id

12315

In [3]:
qs = OcsgeDiff.objects.filter(mpoly__intersects=city.mpoly)
qs = qs.annotate(intersection=Intersection("mpoly", city.mpoly))
qs = qs.annotate(intersection_area=Area(Transform("intersection", 2154)))
qs = qs.values("year_old", "year_new")
qs = qs.annotate(
    new_artif=Sum("intersection_area", filter=Q(is_new_artif=True))
)
qs = qs.annotate(
    new_natural=Sum("intersection_area", filter=Q(is_new_natural=True))
)
qs

<QuerySet [{'year_old': 2015, 'year_new': 2018, 'new_artif': Area(sq_m=35656.28473884208), 'new_natural': None}]>

In [4]:
project = Project.objects.get(pk=6)
project.name

'Diagnostic de Ordan-Larroque'

In [5]:
millesime = Ocsge.objects.filter(mpoly__intersects=project.combined_emprise).latest("year").year
millesime

2019

In [6]:
qs = CommuneSol.objects.filter(city__in=project.cities.all(), year=millesime)
qs = qs.annotate(code_prefix=F("matrix__couverture__code_prefix"))
qs = qs.values("code_prefix")
qs = qs.annotate(surface=Sum("surface"))
qs

<QuerySet [{'code_prefix': 'CS1.1.1.1', 'surface': Decimal('21.9313')}, {'code_prefix': 'CS1.1.1.2', 'surface': Decimal('60.0556')}, {'code_prefix': 'CS1.1.2.1', 'surface': Decimal('2.5660')}, {'code_prefix': 'CS1.2.2', 'surface': Decimal('25.8912')}, {'code_prefix': 'CS2.1.1.1', 'surface': Decimal('806.7629')}, {'code_prefix': 'CS2.1.1.2', 'surface': Decimal('33.1402')}, {'code_prefix': 'CS2.1.1.3', 'surface': Decimal('22.8766')}, {'code_prefix': 'CS2.1.2', 'surface': Decimal('88.8456')}, {'code_prefix': 'CS2.2.1', 'surface': Decimal('3243.8550')}]>

In [7]:
x = PrettyTable(field_names=["code", "label", "Surface", "color"], align="l")

data = list(qs)

for couv in CouvertureSol.objects.all().order_by("code"):
    if couv.level == 2:
        couv.surface = sum([_["surface"] for _ in data if _["code_prefix"].startswith(couv.code_prefix)])
        x.add_row((couv.code, couv.label[:15], couv.surface, couv.map_color))

print(x)

+------+-----------------+-----------+---------+
| code | label           | Surface   | color   |
+------+-----------------+-----------+---------+
| 1.1  | Surfaces anthro | 84.5529   | #ff377a |
| 1.2  | Surfaces nature | 25.8912   | #ccc    |
| 2.1  | Végétation lign | 951.6253  | #80ff00 |
| 2.2  | Végétation non  | 3243.8550 | #ccf24d |
+------+-----------------+-----------+---------+


In [8]:
first_millesime = Ocsge.objects.filter(mpoly__intersects=project.combined_emprise).earliest("year").year
first_millesime

2016

In [9]:
qs = CommuneSol.objects.filter(city__in=project.cities.all(), year__in=[first_millesime, millesime])
qs = qs.annotate(code_prefix=F("matrix__couverture__code_prefix"))
qs = qs.values("code_prefix")
qs = qs.annotate(surface_first=Sum("surface", filter=Q(year=first_millesime)))
qs = qs.annotate(surface_last=Sum("surface", filter=Q(year=millesime)))
qs

<QuerySet [{'code_prefix': 'CS1.1.1.1', 'surface_first': Decimal('21.4977'), 'surface_last': Decimal('21.9313')}, {'code_prefix': 'CS1.1.1.2', 'surface_first': Decimal('59.6789'), 'surface_last': Decimal('60.0556')}, {'code_prefix': 'CS1.1.2.1', 'surface_first': Decimal('2.3400'), 'surface_last': Decimal('2.5660')}, {'code_prefix': 'CS1.2.2', 'surface_first': Decimal('25.8912'), 'surface_last': Decimal('25.8912')}, {'code_prefix': 'CS2.1.1.1', 'surface_first': Decimal('807.1855'), 'surface_last': Decimal('806.7629')}, {'code_prefix': 'CS2.1.1.2', 'surface_first': Decimal('33.1402'), 'surface_last': Decimal('33.1402')}, {'code_prefix': 'CS2.1.1.3', 'surface_first': Decimal('22.8766'), 'surface_last': Decimal('22.8766')}, {'code_prefix': 'CS2.1.2', 'surface_first': Decimal('88.8456'), 'surface_last': Decimal('88.8456')}, {'code_prefix': 'CS2.2.1', 'surface_first': Decimal('3244.4686'), 'surface_last': Decimal('3243.8550')}]>

In [10]:
x = PrettyTable(field_names=["code", "label", "Surface first", "Surface diff", "surface last"], align="l")

data = list(qs)

for couv in CouvertureSol.objects.all().order_by("code"):
    if couv.level == 2:
        couv.surface_first = sum([_["surface_first"] for _ in data if _["code_prefix"].startswith(couv.code_prefix)])
        couv.surface_last = sum([_["surface_last"] for _ in data if _["code_prefix"].startswith(couv.code_prefix)])
        couv.surface_diff = couv.surface_last - couv.surface_first
        x.add_row((couv.code, couv.label[:15], couv.surface_first, couv.surface_diff, couv.surface_last))

print(x)

+------+-----------------+---------------+--------------+--------------+
| code | label           | Surface first | Surface diff | surface last |
+------+-----------------+---------------+--------------+--------------+
| 1.1  | Surfaces anthro | 83.5166       | 1.0363       | 84.5529      |
| 1.2  | Surfaces nature | 25.8912       | 0.0000       | 25.8912      |
| 2.1  | Végétation lign | 952.0479      | -0.4226      | 951.6253     |
| 2.2  | Végétation non  | 3244.4686     | -0.6136      | 3243.8550    |
+------+-----------------+---------------+--------------+--------------+


In [11]:
c = Commune.objects.first()
c

<Commune: L'Abergement-Clémenciat (01001)>

In [12]:
c.departement.source_id

'01'

In [13]:
Commune.objects.get(insee=32301).communediff

AttributeError: 'Commune' object has no attribute 'communediff'

In [15]:
sub_cerema = Cerema.objects.filter(city_insee=OuterRef("insee"))
c = Commune.objects.annotate(
    conso_1121_art=Subquery(sub_cerema.values("naf11art21")[:1]) / 10000,
    conso_1121_hab=Subquery(sub_cerema.values("art11hab21")[:1]) / 10000,
    conso_1121_act=Subquery(sub_cerema.values("art11act21")[:1]) / 10000,
).prefetch_related("communediff_set").get(name="Auch")
c.communediff_set.all()

<QuerySet [<CommuneDiff: CommuneDiff object (13)>]>